# Detect spectral peaks in geophone spectrograms

In [1]:
# Imports
from os.path import join

from pandas import concat
from utils_basic import SPECTROGRAM_DIR as indir, GEO_STATIONS as stations
from utils_spec import assemble_spec_filename, read_geo_spectrograms, find_geo_station_spectral_peaks
from utils_plot import plot_geo_total_psd_and_peaks, save_figure

In [2]:
# Inputs
# Data
window_length = 60.0
overlap = 0.0
downsample = True
downsample_factor = 60

# Finding peaks
rbw_threshold = 0.2
prom_threshold = 5
freqmin = None
freqmax = None


In [3]:
# Loop over days and stations
peak_dfs = []
for station in stations:
    print(f"### Working on {station}... ###")
    
    # Read the spectrograms
    print("Reading the spectrograms")
    if downsample:
        filename = assemble_spec_filename("whole_deployment", "day", "geo", station, window_length, overlap, downsample, downsample_factor = downsample_factor)
        inpath = join(indir, filename)
        stream_spec = read_geo_spectrograms(inpath)
    else:
        filename = assemble_spec_filename("whole_deployment", "day", "geo", station, window_length, overlap, downsample)
        inpath = join(indir, filename)
        stream_spec = read_geo_spectrograms(inpath)

    # Loop over blocks
    
    time_labels = stream_spec.get_time_labels()

    for time_label in time_labels:
        print(f"### Working on {time_label}... ###")
        stream_spec_block = stream_spec.select(time_label = time_label)

        # Find the peaks
        print("Detecting the peaks...")
        peak_df, _ = find_geo_station_spectral_peaks(stream_spec_block, rbw_threshold = rbw_threshold, prom_threshold = prom_threshold, freqmin = freqmin, freqmax = freqmax)
        print(f"In total, {len(peak_df)} spectral peaks found.")

        # Add the station to the dataframe
        peak_df["station"] = station

        # Append to the list
        peak_dfs.append(peak_df)

### Working on 20200113 for A01... ###
Reading the spectrograms
Detecting the peaks...
In total, 3995 spectral peaks found.
### Working on 20200113 for A02... ###
Reading the spectrograms
Detecting the peaks...
In total, 4103 spectral peaks found.
### Working on 20200113 for A03... ###
Reading the spectrograms
Detecting the peaks...
In total, 4417 spectral peaks found.
### Working on 20200113 for A04... ###
Reading the spectrograms
Detecting the peaks...
In total, 5740 spectral peaks found.
### Working on 20200113 for A05... ###
Reading the spectrograms
Detecting the peaks...
In total, 3696 spectral peaks found.
### Working on 20200113 for A06... ###
Reading the spectrograms
Detecting the peaks...
In total, 3899 spectral peaks found.
### Working on 20200113 for A07... ###
Reading the spectrograms
Detecting the peaks...
In total, 4213 spectral peaks found.
### Working on 20200113 for A08... ###
Reading the spectrograms
Detecting the peaks...
In total, 4324 spectral peaks found.
### Work

In [4]:
# Save the results
peak_df = concat(peak_dfs)
peak_df.reset_index(drop = True, inplace = True)

print("Saving the results...")
outdir = indir
filename = f"geo_spectral_peaks_window{window_length:.0f}_overlap{overlap:.1f}_prom{prom_threshold:.0f}_rbw{rbw_threshold:.1f}.csv"
outpath = join(outdir, filename)

peak_df.to_csv(outpath)
print(f"Results saved to {outpath}")

Saving the results...
Results saved to /fp/projects01/ec332/data/spectrograms/daily/daily_geo_spectral_peaks_20200113_window60_overlap0.0_prom5_rbw0.2.csv
